In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:

df = pd.read_csv('train.csv', encoding='ISO-8859-1')

In [3]:
df

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dheeraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [6]:
df.shape

(1599999, 6)

In [7]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
#naming the column
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv('train.csv',names = column_names, encoding='ISO-8859-1')


In [77]:
df['text'][0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [10]:
#counting missing values
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [11]:
#checking the distribution of target columns
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [12]:
#convert label 4 to 1
df.replace({'target':{4:1}}, inplace = True)

In [13]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [19]:
new_df = df.sample(50000, random_state=2)

In [20]:
new_df['target'].value_counts()

target
1    25093
0    24907
Name: count, dtype: int64

### stemming

In [21]:
#stemming
ps=PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]'," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = " ".join(stemmed_content)
    
    return stemmed_content

In [22]:
new_df['stemmed_content'] = new_df['text'].apply(stemming)

In [23]:
new_df.head()

,target,id,date,flag,user,text,stemmed_content
670081,0,2246455751,Fri Jun 19 17:37:19 PDT 2009,NO_QUERY,louiseisanelf,@brodiejay OH IM GOING THERE! Wow Mona Vale is...,brodiejay oh im go wow mona vale real place af...
408251,0,2059364084,Sat Jun 06 16:47:03 PDT 2009,NO_QUERY,MrsAmarieB,my baby's growing up,babi grow
1559739,1,2186151891,Mon Jun 15 18:25:49 PDT 2009,NO_QUERY,epallaviccini,Painted Black-Rolling Stones..the best!,paint black roll stone best
571248,0,2208723981,Wed Jun 17 09:33:02 PDT 2009,NO_QUERY,Kiwitabby,"kk, i'm logging off now BYEZZ!",kk log byezz
524639,0,2193564503,Tue Jun 16 08:37:46 PDT 2009,NO_QUERY,annaqui,Shitty shitty shitty news today,shitti shitti shitti news today


In [58]:
#separiting the data and label
x = new_df['stemmed_content'].values
y = new_df['target'].values

In [59]:
x

array(['brodiejay oh im go wow mona vale real place afteral know suck mvill slow train pffft',
       'babi grow', 'paint black roll stone best', ...,
       'jonoh care bot easi retweet peopl slag brand rather say nice thing',
       'sleep lto laterrr love friendster mani chismi hahahaha good mornight tweepl game hr',
       'desper need inspir definit lack today'], dtype=object)

In [60]:
y

array([0, 0, 1, ..., 1, 1, 0])

In [61]:
#splitting data to train and test
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, stratify=y, random_state=2)

In [62]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [63]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 296781 stored elements and shape (40000, 40615)>
  Coords	Values
  (0, 8270)	0.43821477846789586
  (0, 28222)	0.2594278408771296
  (0, 1984)	0.3096593404945773
  (0, 12363)	0.34184279368540876
  (0, 37526)	0.43821477846789586
  (0, 33165)	0.3072710990070114
  (0, 23200)	0.40138883470369746
  (0, 10813)	0.28347642509130505
  (1, 18028)	0.6333253287632269
  (1, 18023)	0.6429092059819265
  (1, 13496)	0.4307746287917145
  (2, 35651)	0.4226702697071805
  (2, 16132)	0.6022100982331299
  (2, 13617)	0.32972597486374244
  (2, 39421)	0.3731734659170678
  (2, 36196)	0.4590372387152281
  (3, 22844)	0.4087881934597973
  (3, 5329)	0.4087881934597973
  (3, 26662)	0.3308269725085545
  (3, 17106)	0.34487476284811225
  (3, 18523)	0.32246100339657907
  (3, 17485)	0.3308269725085545
  (3, 8360)	0.27827155120315683
  (3, 383)	0.382801119809383
  (4, 27585)	0.30309574461203787
  :	:
  (39996, 6731)	0.2677982067232491
  (39996, 38457)	0.1762622239

In [64]:
#logistic regression
model = LogisticRegression(max_iter = 1000)

In [65]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [66]:
#accuracy score
x_train_pred= model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_pred)

In [67]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.835125


In [68]:
#accuracy score
x_test_pred= model.predict(x_test)
test_data_accuracy = accuracy_score(y_test, x_test_pred)
print('Accuracy score on the test data :', test_data_accuracy)

Accuracy score on the test data : 0.7539


In [76]:
import pickle
#pickle.dump(vectorizer,open('vectorizer.pkl','wb'))
#pickle.dump(model,open('model.pkl','wb'))